In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score

In [ ]:
from google.colab import drive #connecting to drive to import files
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
#importing test file
test_url = 'https://drive.google.com/uc?id={}'.format('1fyfT8LH8yksUqfAkeDvGmk2Sg4KLQwSt')
test = pd.read_csv(test_url)
test
#https://drive.google.com/file/d/1fyfT8LH8yksUqfAkeDvGmk2Sg4KLQwSt/view?usp=share_link

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S
...,...,...,...,...,...,...,...,...,...,...,...
413,1305,3,"Spector, Mr. Woolf",male,NaN,0,0,A.5. 3236,8.0500,NaN,S
414,1306,1,"Oliva y Ocana, Dona. Fermina",female,39.0,0,0,PC 17758,108.9000,C105,C
415,1307,3,"Saether, Mr. Simon Sivertsen",male,38.5,0,0,SOTON/O.Q. 3101262,7.2500,NaN,S
416,1308,3,"Ware, Mr. Frederick",male,NaN,0,0,359309,8.0500,NaN,S


In [ ]:
#importing train file
train_url = 'https://drive.google.com/uc?id={}'.format('1z-1Y9iBKifjHbNFWW4JK7VugIZtlmORz')
train = pd.read_csv(train_url)
train
#https://drive.google.com/file/d/1z-1Y9iBKifjHbNFWW4JK7VugIZtlmORz/view?usp=share_link

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
...,...,...,...,...,...,...,...,...,...,...,...,...
886,887,0,2,"Montvila, Rev. Juozas",male,27.0,0,0,211536,13.0000,NaN,S
887,888,1,1,"Graham, Miss. Margaret Edith",female,19.0,0,0,112053,30.0000,B42,S
888,889,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,NaN,1,2,W./C. 6607,23.4500,NaN,S
889,890,1,1,"Behr, Mr. Karl Howell",male,26.0,0,0,111369,30.0000,C148,C


ideas for most important columns:
- age (children and elderly)
- pclass (1 = 1st class ticket, the lower the number the richer)
- sex (women first, women alone predicted 74% accuracy)
-  fare


to-do:
- deal with na's
- pick features we want to use
- pick a classifier system
- split train into train and validation
- train on the training set
- test on test set
- find accuracy
- once its above 0.74850, turn predictions into csv file and submit

In [ ]:
test.info() #age and cabin have few null values

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 418 entries, 0 to 417
Data columns (total 11 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  418 non-null    int64  
 1   Pclass       418 non-null    int64  
 2   Name         418 non-null    object 
 3   Sex          418 non-null    object 
 4   Age          332 non-null    float64
 5   SibSp        418 non-null    int64  
 6   Parch        418 non-null    int64  
 7   Ticket       418 non-null    object 
 8   Fare         417 non-null    float64
 9   Cabin        91 non-null     object 
 10  Embarked     418 non-null    object 
dtypes: float64(2), int64(4), object(5)
memory usage: 36.0+ KB


In [ ]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB


In [ ]:
from re import X
import pandas as pd
from sklearn.neural_network import MLPClassifier
from sklearn.svm import SVC

from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.feature_extraction import DictVectorizer

from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV, ParameterGrid

import numpy as np

import warnings
warnings.filterwarnings("ignore")

train['Age'].fillna(train['Age'].mean(), inplace=True)
test['Age'].fillna(test['Age'].mean(), inplace=True)
test['Fare'].fillna(test['Fare'].mean(), inplace=True)
train['Embarked'].fillna(train['Embarked'].value_counts().index[0], inplace=True)
test['Embarked'].fillna(test['Embarked'].value_counts().index[0], inplace=True)

titles = train['Name'].str.extract(' ([A-Za-z]+)\.', expand=False)
titles.unique()

titles = test['Name'].str.extract(' ([A-Za-z]+)\.', expand=False)
titles.unique()

#we watched numerous YouTube videos and read multiple blogs to better understand dataset and get hints on useful feature engineering
#we also used chatGPT to refine our code and get some helpful small code snippets

#notice changing the key's name also affect accuracy

#Titles can give us info about demographic information, such as age, gender, marital and social status.
def extract_title(df):
    map_titles = {'Mr': 'Mr', 'Master': 'Master', 'Mrs': 'Mrs', 'Ms': 'Mrs', 'Mme': 'Mrs', 'Miss': 'Miss',
                  'Mlle': 'Miss', 'Capt': 'Officer', 'Col': 'Officer', 'Major': 'Officer', 'Dr': 'Officer',
                  'Rev': 'Officer', 'Jonkheer': 'Royalty', 'Don': 'Royalty', 'Sir': 'Royalty',
                  'Countess': 'Royalty', 'Lady': 'Royalty'}
    df['Title'] = df['Name'].str.extract(' ([A-Za-z]+)\.', expand=False).map(map_titles)

extract_title(train)
extract_title(test)

#family members with same last name and ticket number can survive or die together
def extract_family(df):
    last_name = df['Name'].str.extract('(.+?), ', expand=False)
    df['Family'] = last_name + '_' + df['Ticket'].str[:-1]

extract_family(train)
extract_family(test)

# Data encoding
train['Pclass'] = train['Pclass'].apply(lambda x: (x - np.average(train['Pclass']))\
                                            / np.std(train['Pclass']))
test['Pclass'] = test['Pclass'].apply(lambda x: (x - np.average(test['Pclass']))\
                                            / np.std(test['Pclass']))

#lets try age and fare - got less accuracy in test dataset
'''
train['Age'] = train['Age'].apply(lambda x: (x - np.average(train['Age']))\
                                            / np.std(train['Age']))
test['Age'] = test['Age'].apply(lambda x: (x - np.average(test['Age']))\
                                            / np.std(test['Age']))

train['Fare'] = train['Fare'].apply(lambda x: (x - np.average(train['Fare']))\
                                            / np.std(train['Fare']))
test['Fare'] = test['Fare'].apply(lambda x: (x - np.average(test['Fare']))\
                                            / np.std(test['Fare']))

'''
def get_dummies(train, test, columns):
    df = pd.concat([train[columns], test[columns]])
    df = pd.get_dummies(df)
    xtrain = df.iloc[:train.shape[0]]
    xtest = df.iloc[train.shape[0]:]
    return xtrain, xtest


#added two new features: Title & Family - gives better accuracy

features = ['Pclass', 'Sex', 'SibSp', 'Parch', 'Embarked', 'Title', 'Family']
xtrain, xtest = get_dummies(train, test, features)
ytrain = train['Survived']

# Model building and evaluation
create_params = {'max_iter': 1000, 'random_state': 0,'alpha':1, 'hidden_layer_sizes': 50, 'learning_rate_init': 0.002, 'batch_size': 256}
model = MLPClassifier(**create_params)


model.fit(xtrain, ytrain)
ypred = model.predict(xtrain)

# Calculate accuracy
accuracy = accuracy_score(ypred, ytrain)
accuracy







0.9887766554433222

In [ ]:
submission = pd.DataFrame(index=test.PassengerId)
submission['Survived'] = model.predict(xtest)

In [ ]:
submission.reset_index().to_csv('submission99999.csv', index=False)

In [ ]:
submission

,Survived
PassengerId,
892,0
893,1
894,0
895,0
896,1
...,...
1305,0
1306,1
1307,0
